In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from IPython.display import display, HTML
warnings.simplefilter(
    action='ignore', category=display)

In [70]:
# circuits = pd.read_csv("./rawdata/circuits.csv")
# constructor_results = pd.read_csv("./rawdata/constructor_results.csv")
# constructor_standings = pd.read_csv("./rawdata/constructor_standings.csv")
# constructors = pd.read_csv("./rawdata/constructors.csv")
# driver_standings = pd.read_csv("./rawdata/driver_standings.csv")
# drivers = pd.read_csv("./rawdata/drivers.csv")
# lap_times = pd.read_csv("./rawdata/lap_times.csv")
# pit_stops = pd.read_csv("./rawdata/pit_stops.csv")
# qualifying = pd.read_csv("./rawdata/qualifying.csv")
# races = pd.read_csv("./rawdata/races.csv")
# results = pd.read_csv("./rawdata/results.csv")
# seasons = pd.read_csv("./rawdata/seasons.csv")
# sprint_results = pd.read_csv("./rawdata/sprint_results.csv")
# status = pd.read_csv("./rawdata/status.csv")


# TEAM COLORS
TEAM_COLORS = {
    "mercedes": "#27F4D2",
    "red_bull": "#3671C6",
    "ferrari": "#E80020",
    "mclaren": "#FF8000",
    "alpine": "#00A1E8",
    "rb": "#6692FF",
    "aston_martin": "#229971",
    "williams": "#1868DB",
    "sauber": "#52E252",
    "haas": "#B6BABD",
}

TABLE_COLORS = {
    "header": "#e90d18",
    "fastest": "#a528c2",
    "sub": "#f0cc38",
    "gray": "#dddddd",
    "black": "#000000"
}

# Data Preprocessing


In [ ]:
# # drop unneccessary columns
# drivers.drop(columns='url')
# circuits.drop(columns='url')
# constructors.drop(columns='url')
# # add fullname column in 'drivers'
# drivers["fullname"] = drivers["forename"] + " " + drivers["surname"]

# # convert milliseconds to seconds


# def convert_to_seconds(val):
#     try:
#         return int(val) / 6000
#     except ValueError:
#         return val


# drivers.to_csv("./data/drivers.csv", index=False)
# results.to_csv("./data/results.csv", index=False)
# sprint_results.to_csv("./data/sprint_results.csv", index=False)

# Data Manipulation

## Race and Championship Results

In [72]:
class F1Query:

    class NoResultError(Exception):
        def __init__(self, exact=True):
            self.exact = exact
            if exact:
                super().__init__("No result found. Enter valid keywords")
            else:
                super().__init__(
                    "No result found. Try 'exact=False' or 'F1Query.setExact(False)'.")

    class MultipleResultError(Exception):
        def __init__(self, keyword):
            self.keyword = keyword
            super().__init__(
                f"Multiple result found with keyword[{keyword}]. Try more complicated keywords")

    def __init__(self):
        self.circuits = pd.read_csv("./data/circuits.csv", index_col=False)
        self.constructor_results = pd.read_csv(
            "./data/constructor_results.csv", index_col=False)
        self.constructor_standings = pd.read_csv(
            "./data/constructor_standings.csv", index_col=False)
        self.constructors = pd.read_csv(
            "./data/constructors.csv", index_col=False)
        self.driver_standings = pd.read_csv(
            "./data/driver_standings.csv", index_col=False)
        self.drivers = pd.read_csv("./data/drivers.csv",  index_col=False)
        self.lap_times = pd.read_csv("./data/lap_times.csv",  index_col=False)
        self.pit_stops = pd.read_csv("./data/pit_stops.csv",  index_col=False)
        self.qualifying = pd.read_csv(
            "./data/qualifying.csv",  index_col=False)
        self.races = pd.read_csv("./data/races.csv",  index_col=False)
        self.results = pd.read_csv("./data/results.csv",  index_col=False)
        self.seasons = pd.read_csv("./data/seasons.csv",  index_col=False)
        self.sprint_results = pd.read_csv(
            "./data/sprint_results.csv",  index_col=False)
        self.status = pd.read_csv("./data/status.csv",  index_col=False)

        self.__exact = False
        self.__verbose = True

        self._font_init()

    def _font_init(self):
        display(HTML("<style>table {font-family: 'Times New Roman';}</style>"))

    def setExact(self, exactness):
        assert type(exactness) == bool
        self.__exact = exactness

    def setVerbose(self, verboseness):
        assert type(verboseness) == bool
        self.__verbose = verboseness

    def getCircuitByName(self, circuitName: str, exact=None) -> pd.DataFrame:
        if exact is None:
            exact = self.__exact
        circuitName = circuitName.title()
        circuitQuery = self.circuits.query(
            "name == @circuitName") if exact else self.circuits.query(
            "name.str.contains(@circuitName, case=False) or circuitRef.str.contains(@circuitName, case=False)")
        if len(circuitQuery) == 0:
            raise self.NoResultError(exact)
        else:
            return circuitQuery

    def getCircuitByCountry(self, country: str, exact=None) -> pd.DataFrame:
        if exact is None:
            exact = self.__exact
        country = country.title()
        circuitQuery = self.circuits.query(
            "country == @country") if exact else circuits.query(
            "country.str.contains(@country, case=False)")
        if len(circuitQuery) == 0:
            raise self.NoResultError(exact)
        else:
            return circuitQuery

    def getCircuitByKeyword(self, keyword: str, exact=None) -> pd.DataFrame:
        keyword = keyword.title()
        if exact is None:
            exact = self.__exact
        circuitQuery = []

        if exact:
            circuitQuery = self.circuits.query(
                "name == @keyword or country == @keyword or location == @keyword")
        else:
            circuitQuery = self.circuits.query(
                "name.str.contains(@keyword, case=False) or country.str.contains(@keyword, case=False) or location.str.contains(@keyword, case=False) or circuitRef.str.contains(@keyword, case=False)")

        if len(circuitQuery) == 0:
            raise self.NoResultError(exact)
        else:
            return circuitQuery

    def getRacesByKeyword(self, keyword: str, exact=None) -> pd.DataFrame:
        if exact is None:
            exact = self.__exact
        circuitIds = list(self.getCircuitByKeyword(
            keyword, exact)["circuitId"])
        raceQuery = self.races.query("circuitId in @circuitIds")
        if len(raceQuery) == 0:
            raise self.NoResultError(exact)
        return raceQuery

    def getRacesByYear(self, year: int) -> pd.DataFrame:
        query = self.races.query("year == @year")
        if len(query) == 0:
            raise self.NoResultError(exact)
        return query

    def getRaceIdByYearKeyword(self, year: int, keyword: str, exact=None) -> int:
        if exact is None:
            exact = self.__exact
        raceQuery = self.getRacesByKeyword(
            keyword, exact).query("year == @year")

        if len(raceQuery) == 0:
            raise self.NoResultError(exact)
        elif len(raceQuery) > 1:
            raise self.MultipleResultError(keyword)
        else:
            return raceQuery.iloc[0].raceId

    def getRaceIdByYearRound(self, year: int, round: int) -> int:
        race = self.races.query("year == @year and round == @round")
        if len(race) == 0:
            raise self.NoResultError(exact)
        return race.iloc[0].raceId

    def getDriverIdByNumber(self, driverNumber) -> int:
        driverQuery = self.drivers.query("number == @driverNumber")
        if len(driverQuery) == 0:
            self.NoResultError()
        return driverQuery.iloc[0].driverId

    def getDriverByName(self, driverName: str) -> pd.DataFrame:
        driverName = driverName.title()
        driverQuery = self.drivers.query(
            "fullname.str.contains(@driverName, case=False) or driverRef.str.contains(@driverName, case=False)")
        if len(driverQuery) == 0:
            self.NoResultError()
        return driverQuery

    def getRaceResultById(self, raceId) -> pd.DataFrame:
        raceResults = self.results.query("raceId == @raceId")
        if len(raceResults) == 0:
            raise self.NoResultError()
        return raceResults

    def getRaceResultByKeyword(self, keyword, exact=None):
        if exact is None:
            exact = self.__exact
        races = self.getCircuitByKeyword(keyword, exact)

# Visualization 

## Visualize Race
## Visualize Driver's Championship
## Visualize Constructor's Championship

In [ ]:
class F1Visualizer:
    def __init__(self):
        Query = F1Query()
        self.colors = TEAM_COLORS
        self.font = "Formula1"
        self.query = F1Query()
        self.circuits = pd.read_csv("./data/circuits.csv", index_col=False)
        self.constructor_results = pd.read_csv(
            "./data/constructor_results.csv", index_col=False)
        self.constructor_standings = pd.read_csv(
            "./data/constructor_standings.csv", index_col=False)
        self.constructors = pd.read_csv(
            "./data/constructors.csv", index_col=False)
        self.driver_standings = pd.read_csv(
            "./data/driver_standings.csv", index_col=False)
        self.drivers = pd.read_csv("./data/drivers.csv",  index_col=False)
        self.lap_times = pd.read_csv("./data/lap_times.csv",  index_col=False)
        self.pit_stops = pd.read_csv("./data/pit_stops.csv",  index_col=False)
        self.qualifying = pd.read_csv(
            "./data/qualifying.csv",  index_col=False)
        self.races = pd.read_csv("./data/races.csv",  index_col=False)
        self.results = pd.read_csv("./data/results.csv",  index_col=False)
        self.seasons = pd.read_csv("./data/seasons.csv",  index_col=False)
        self.sprint_results = pd.read_csv(
            "./data/sprint_results.csv",  index_col=False)
        self.status = pd.read_csv("./data/status.csv",  index_col=False)
        self.driverDict = dict(
            zip(self.drivers["driverId"], self.drivers["code"]))
        self.constructorDict = dict(
            zip(self.constructors["constructorId"], zip(self.constructors["name"], self.constructors["constructorRef"])))
        self.status = dict(
            zip(self.status["statusId"], self.status["status"]))
        self.HEADER_STYLE = [
            {'selector': 'th', 'props': [
                ('font-weight', 'Display-Bold'), ('color', TABLE_COLORS['header']), ('background-color', TABLE_COLORS['gray'])]},
            # Italicize all cells
            # {'selector': 'td', 'props': [('color', TABLE_COLORS['black'])]}
        ]
        self._font_init()

    def _font_init(self):
        display(HTML("<style>table {font-family: 'Formula1';}</style>"))

    # Replace IDs to readable strings
    def _addReadables(self, df):
        try:
            df['DRIVER'] = list(
                map(lambda x: self.driverDict[x], df['driverId']))
        except KeyError:
            pass
        try:
            df['TEAM'] = list(
                map(lambda x: self.constructorDict[x][0], df['constructorId']))
        except KeyError:
            pass
        try:
            df['STATUS'] = list(map(lambda x: self.status[x], df['statusId']))
        except KeyError:
            pass
        return df

    def _teamColor(self, row, index=1):
        color = TEAM_COLORS.get(self.constructorDict.get(
            row.constructorId)[1], "black")
        return [''] * index + [f'color: {color}'] * 2 + [''] * (len(row) - index - 2)

    def _podiumColor(self, val):
        color = ""
        try:
            val = int(val)
        except ValueError:
            pass

        if val == 1:
            color = f"{TABLE_COLORS['fastest']}; font-weight: bold"
        elif val == 2 or val == 3:
            color = f"{TABLE_COLORS['sub']}; font-weight: bold"
        elif val == "R" or val == "N":
            color = f"{TABLE_COLORS['header']}; font-weight: bold"
        else:
            color = ""
        return f'color: {color}'

    def _statusColor(self, val):
        color = ""
        if val == "Finished":
            color = ""
        elif "Lap" in val:
            color = TABLE_COLORS['sub']
        else:
            color = TABLE_COLORS['header']
        return f'color: {color}'

    def _deltaColor(self, val):
        color = ""
        if val == "-":
            color = ""
        elif int(val) > 0:
            color = TABLE_COLORS['sub']
        else:
            color = TABLE_COLORS['fastest']
        return f'color: {color}'

    def _minColor(self, s):
        s = s.replace("-", chr(0x10ffff))
        minVal = s.min()
        print(minVal)
        return [f"color: {TABLE_COLORS['fastest']}; font-weight: bold" if v == minVal else '' for v in s]

    def _posDelta(self, row):
        if row['positionText'] in 'NR':
            return "-"
        else:
            try:
                int(row['grid'])
            except ValueError:
                return "-"
            delta = int(row['positionText']) - int(row['grid'])
            if delta > 0:
                return "+" + str(delta)
            elif delta == 0:
                return "-"
            else:
                return str(delta)

    def showRaceResult(self, raceId):
        # get race results
        raceResult = self.query.getRaceResultById(raceId).copy()
        raceResult = self._addReadables(raceResult)

        # drop unneccesarry columns and replace special values
        raceResult.astype(str)
        raceResult.drop(columns=['resultId', 'raceId', 'milliseconds',
                        'fastestLap', 'rank', 'fastestLapSpeed', 'position', 'positionOrder', 'number'], inplace=True)
        raceResult['points'] = raceResult.points.astype(int)
        raceResult['time'].replace("\\N", "-", inplace=True)
        raceResult['fastestLapTime'].replace("\\N", "-", inplace=True)
        raceResult['∆POS'] = raceResult.apply(self._posDelta, axis=1)
        # rename and reindex
        raceResult.rename(columns={
            "positionText": "POS",
            "grid": "GRID",
            "points": "PTS",
            "time": "TIME",
            "fastestLapTime": "FASTEST LAP",
            "laps": "LAPS"
        }, inplace=True)

        raceResult = raceResult[["POS", "DRIVER", "TEAM", "LAPS",
                                 "TIME", "PTS", "FASTEST LAP", "GRID", "∆POS", "STATUS", "driverId", "constructorId", "statusId"]]

        # styling
        styledResult = raceResult.style.hide(  # hide columns
            ["driverId", "constructorId", "statusId"], axis=1
        ).hide(

        ).set_table_styles(
            self.HEADER_STYLE  # set header style
        ).applymap(
            self._podiumColor, subset=['POS']
        ).applymap(
            self._statusColor, subset=['STATUS']
        ).applymap(
            self._deltaColor, subset=['∆POS']
        ).apply(
            self._teamColor, axis=1
        ).apply(
            self._minColor, subset=['FASTEST LAP']
        )
        return styledResult

In [159]:
Query = F1Query()
Visualizer = F1Visualizer()

In [160]:
Visualizer.showRaceResult(1122)

1:31.632


POS,DRIVER,TEAM,LAPS,TIME,PTS,FASTEST LAP,GRID,∆POS,STATUS
1,VER,Red Bull,50,1:20:43.273,25,1:31.773,1,-,Finished
2,PER,Red Bull,50,+13.6431,18,1:32.273,3,-1,Finished
3,LEC,Ferrari,50,+18.639,16,1:31.632,2,+1,Finished
4,PIA,McLaren,50,+32.007,12,1:32.310,5,-1,Finished
5,ALO,Aston Martin,50,+35.759,10,1:32.387,4,+1,Finished
6,RUS,Mercedes,50,+39.936,8,1:32.254,7,-1,Finished
7,BEA,Ferrari,50,+42.679,6,1:32.186,11,-4,Finished
8,NOR,McLaren,50,+45.708,4,1:31.944,6,+2,Finished
9,HAM,Mercedes,50,+47.391,2,1:31.746,8,+1,Finished
10,HUL,Haas F1 Team,50,+1:16.996,1,1:32.366,15,-5,Finished


In [104]:
raceResult = Query.getRaceResultById(1122)

In [105]:
df = raceResult[['positionText', 'grid']]
df = df.astype(str)

In [ ]:
def _posDelta(row):

    if row['positionText'] in 'NR':
        return "-"
    else:
        try:
            int(row['grid'])
        except ValueError:
            return "-"
        delta = int(row['positionText']) - int(row['grid'])
        if delta > 0:
            return "+" + str(delta)
        else:
            return str(delta)

In [107]:
df['∆GRID'] = df.apply(_posDelta, axis=1)

In [108]:
df

,positionText,grid,∆GRID
26300,1,1,0
26301,2,3,-1
26302,3,2,+1
26303,4,5,-1
26304,5,4,+1
26305,6,7,-1
26306,7,11,-4
26307,8,6,+2
26308,9,8,+1
26309,10,15,-5


In [82]:
df.dtypes

positionText    object
grid            object
dtype: object

True

In [145]:
"1124124" > "9"

False

In [ ]:
chr(0x10ffff)

True